In [ ]:
import os

from dotenv import load_dotenv
import pymysql
from password_generator import PasswordGenerator

pwo = PasswordGenerator()

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

users = []

with pymysql.cursors.Cursor(con) as cursor:
    cursor.execute("SELECT empId, ,employeeDeploySite FROM employees")
    employees = cursor.fetchall()

    for employee in employees:
        # print("🐷", employee)
        users.append([employee[0], pwo.shuffle_password("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789@", 5), employee[1]])

    with pymysql.cursors.Cursor(con) as cursor:
        for user in users:
            if user[2] is None: user[2] = 'None'
            print("🐷 oink", user)
            cursor.execute("INSERT INTO attendance_users (user_id, password, site_location) VALUES (%s, %s, %s)", user)
            con.commit()


In [ ]:
import csv

with open("users.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["user_id", "password", "site_location"])
    for user in users:
        writer.writerow(user)


# Read csv data and update with contact from Database


In [ ]:
import os
import csv

from dotenv import load_dotenv
import pymysql

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

new_users = []

with open("users.csv") as file:
    reader = csv.reader(file)
    next(reader) # ignore header

    for line in reader:
        print("🐖 oink", line)
        with pymysql.cursors.Cursor(con) as khinzeer:
            khinzeer.execute("SELECT contact from employees WHERE empId = %s", line[0])
            contact = khinzeer.fetchone()[0]
            line.append(contact)
            new_users.append(line)

with open("users1.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["user_id", "password", "site_location", "contact"])
    for user in new_users:
        writer.writerow(user)
